### TabTransformer

The TabTransformer architecture works as follows:

1. All the categorical features are encoded as embeddings, using the same `embedding_dims`. <font color=red><b>This means that each value in each categorical feature will have its own embedding vector.</b></font>
2. **A column embedding**, one embedding vector for each categorical feature, is added (point-wise) to the categorical feature embedding.
3. The embedded categorical features are fed into a <font color=red><b>stack of Transformer blocks</b></font>. Each Transformer block consists of a **multi-head self-attention layer followed by a feed-forward layer**.
4. The outputs of the final Transformer layer, which are the contextual embeddings of the categorical features, are concatenated with the input numerical features, and fed into a final **Multi-layer perceptron (MLP) block.**
5. A `sigmoid` classifer is applied at the end of the model, since we're solving a **binary classifier problem.**

The following image Illustrates the structure behind each <font color=red><b>stack of Transformer block.</b></font>

<img src="https://raw.githubusercontent.com/keras-team/keras-io/master/examples/structured_data/img/tabtransformer/tabtransformer.png" alt="tabtransformer_model" width="400px"/>

In [5]:
from __future__ import annotations

from typing import Any, Dict, List, Tuple, Union

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from multipledispatch import dispatch
from tensorflow import keras
import itertools
import hydra
import tensorflow as tf
from rich.console import Console


In [7]:
console = Console()

In [6]:
train = pd.read_csv('train_data.csv')

In [8]:
def str_to_list(str_value: str | List[str] | None = None) -> List[Union[str, None]]:
    """
    Convert parameter :param:`str_value` to a list if it is a string.

    Parameters
    ----------
    str_value : str | List[str] | None, optional
        String or list of strings. If None, return an empty list.

    Returns
    -------
    List[Union[str, None]]
        List of strings or empty list.

    Raises
    ------
    ValueError
        If :param:`str_value` is not a string or a list of strings, or None.
    """
    if isinstance(str_value, str):
        return [str_value]
    if isinstance(str_value, list):
        return str_value
    if str_value is None:
        return []
    raise ValueError(
        f"The parameter 'str_value' must be a string or a list of strings or "
        f"None. Parameter of type '<{type(str_value)}>' was passed."
    )


def get_categorical_features_vocab(
    df: pd.DataFrame,
    categorical_features_names: List[Union[str, None]] | str,
) -> Dict[str, List[str]] | Dict:
    """
    Get the vocabulary of the categorical features.

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe to be encoded. Dataframe is where the categorical features
        unique vocabularies are extracted from.
    categorical_features_names : List[Union[str, None]] | str
        List of categorical features names, or a string with the name of a
        categorical feature, or an empty list, if no categorical features
        exist.

    Returns
    -------
    Dict[str, List[str]] | Dict
        Dictionary with the categorical features names as keys and the
        categorical features unique vocabularies as values. If no categorical
        features exist, return an empty dictionary.
    """
    categorical_features_names = str_to_list(categorical_features_names)
    if categorical_features_names:
        return {
            categorical_feature_name: sorted(
                list(df[categorical_feature_name].unique())
            )
            for categorical_feature_name in categorical_features_names
        }
    return {}


def create_model_inputs(
    feature_names: List[str], numeric_features_names: List[str]
) -> Dict[str, tf.Tensor]:
    
    _inputs = {}
    for feature_name in feature_names:
        if feature_name in numeric_features_names:
            _inputs[feature_name] = tf.keras.layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            _inputs[feature_name] = tf.keras.layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return _inputs


def encode_inputs(
    inputs,
    embedding_dims,
    categorical_features_names,
    categorical_features_vocabulary,
):

    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in categorical_features_names:

            # Get the vocabulary of the categorical feature.
            vocabulary = categorical_features_vocabulary[feature_name]

            # Create a lookup to convert string values to an integer indices.
            # Since we aren't using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = tf.keras.layers.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int",
            )

            # Convert the string input values into integer indices.
            encoded_feature = lookup(inputs[feature_name])

            # Create an embedding layer with the specified dimensions.
            embedding = tf.keras.layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(encoded_feature)
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:

            # Use the numerical features as-is.
            numerical_feature = tf.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list


def create_mlp(
    hidden_units: List[int],
    dropout_rate: float,
    activation: str,
    normalization_layer,
    name: str | None = None,
):

    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer),
        mlp_layers.append(tf.keras.layers.Dense(units, activation=activation))
        mlp_layers.append(tf.keras.layers.Dropout(dropout_rate))

    return tf.keras.Sequential(mlp_layers, name=name)


def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    inputs: Dict[str, tf.Tensor],
    categorical_features_names,
    categorical_features_vocabulary,
    use_column_embedding=False,
):

    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs,
        embedding_dims,
        categorical_features_names,
        categorical_features_vocabulary,
    )

    # Stack categorical feature embeddings for the Transformer.
    encoded_categorical_features = tf.stack(encoded_categorical_feature_list, axis=1)

    # Concatenate numerical features.
    numerical_features = tf.keras.layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = tf.keras.layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        column_indices = tf.range(start=0, limit=num_columns, delta=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create many layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        
        # Create a multi-head attention layer.
        attention_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)

        # Skip connection 1.
        x = tf.keras.layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )

        # Layer normalization 1.
        x = tf.keras.layers.LayerNormalization(
            name=f"layer_norm1_{block_idx}", epsilon=1e-6
        )(x)

        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=tf.keras.activations.gelu,
            normalization_layer=tf.keras.layers.LayerNormalization(epsilon=1e-6),
            name=f"feedforward_{block_idx}",
        )(x)

        # Skip connection 2.
        x = tf.keras.layers.Add(name=f"skip_connection2_{block_idx}")(
            [feedforward_output, x]
        )

        # Layer normalization 2.
        encoded_categorical_features = tf.keras.layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)

    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = tf.keras.layers.Flatten()(encoded_categorical_features)

    # Apply layer normalization to the numerical features.
    numerical_features = tf.keras.layers.LayerNormalization(epsilon=1e-6)(
        numerical_features
    )

    # Prepare the input for the final MLP block.
    features = tf.keras.layers.concatenate([categorical_features, numerical_features])

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    
    # Create final "MLP" - Multi Layer Perceptron.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=tf.keras.activations.selu,
        normalization_layer=tf.keras.layers.BatchNormalization(),
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifier.
    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid", name="sigmoid")(
        features
    )
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def prepare_example(features, target, **kwargs: Any):
    mask_token = kwargs.get("mask_token", None)
    num_oov_indices = kwargs.get("num_oov_indices", 0)
    
    target_label_lookup = tf.keras.layers.StringLookup(
        vocabulary=TARGET_FEATURE_LABELS, mask_token=mask_token, num_oov_indices=num_oov_indices
    )
    target_index = target_label_lookup(target)
    weights = features.pop(WEIGHT_COLUMN_NAME)
    return features, target_index, weights


def run_experiment(model, dataset, num_epochs, learning_rate, weight_decay):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
    )

    callbacks_list = [
        tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2),
        tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoint_path.keras", monitor="val_loss", save_best_only=True
        ),
        tf.keras.callbacks.TensorBoard(log_dir=r"./logs"),
    ]

    console.print("Start training the model...")
    _history = model.fit(dataset, epochs=num_epochs, callbacks=callbacks_list)
    console.print("[bold green]Model training finished![/bold green]")

    return _history, model


def df_to_dataset(
    dataframe: pd.DataFrame,
    target_feature_name: str | None = None,
    shuffle: bool = True,
    batch_size: int = 32,
) -> tf.data.Dataset:
    """Convert a Pandas DataFrame to a TensorFlow Dataset.

    Parameters
    ----------
    dataframe : pd.DataFrame
        Pandas DataFrame to be converted to a TensorFlow Dataset.
    target_feature_name : str | None, optional
        Name of the target feature. If `None`, the target feature is not
        included in the dataset.
    shuffle : bool, optional
        Whether to shuffle the dataset. Defaults to `True`.
    batch_size : int, optional
        Batch size. Defaults to 32.

    Returns
    -------
    tf.data.Dataset
        TensorFlow Dataset.

    .. seealso::

        * `tf.data.Dataset`: `https://www.tensorflow.org/api_docs/python/tf/data/Dataset`_

    """
    df = dataframe.copy()

    labels = None

    if isinstance(target_feature_name, str) and target_feature_name in df.columns:
        labels = df.pop(target_feature_name)

    df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        ds = ds.prefetch(batch_size)
    return ds



In [9]:
train_data_file = r"./train.csv"
train_data = pd.read_csv(train_data_file)

TARGET_FEATURE_NAME = "FLG_ACAO_EFICAZ"
TARGET_FEATURE_LABELS = ["0", "1"]
NUM_CLASSES = len(TARGET_FEATURE_LABELS)
NUMERIC_FEATURE_NAMES = [
    "DIAS_ENTRE_ACAO_VENC",
    "HST_DIAS_ENTRE_ACAO_PGTO",
    "IDADE_CONTA",
    "VLR_FATURA",
    "ZCGDOCREF",
]
CATEGORICAL_FEATURE_NAMES = [
    "BAIXARENDA",
    "CLASSE",
    "FORM_PAGT",
    "FORNECEDOR",
    "TARIFA",
    "ZCGTAXTYP",
    "ZCGTIPLOC",
]

WEIGHT_COLUMN_NAME = "FAKE_WEIGHT_FEATURE"
train_data = train_data.astype(
    {
        **{num_feat: float for num_feat in NUMERIC_FEATURE_NAMES},
        **{cat_feat: str for cat_feat in CATEGORICAL_FEATURE_NAMES},
        **{target_feat: str for target_feat in [TARGET_FEATURE_NAME]},
    }
)
CATEGORICAL_FEATURES_WITH_VOCABULARY = get_categorical_features_vocab(
    train_data, CATEGORICAL_FEATURE_NAMES
)
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
CSV_HEADER = FEATURE_NAMES + [TARGET_FEATURE_NAME, WEIGHT_COLUMN_NAME]
COLUMN_DEFAULTS = [
    [0] if feature_name in NUMERIC_FEATURE_NAMES + [TARGET_FEATURE_NAME] else ["NA"]
    for feature_name in CSV_HEADER
]
train_data[WEIGHT_COLUMN_NAME] = 1.0


In [10]:
# Create model inputs.
inputs = create_model_inputs(feature_names=FEATURE_NAMES, numeric_features_names=NUMERIC_FEATURE_NAMES)

### Building the Model

After defining all necessary functions, it's time time to start building the model. At the end of the next cell,
after building the model, the following code print the graph structure of our generated model. If you find the image too small to see,
you can open the image outside Jupyter. The model structure is saved as `tabtransformer.png`, inside the same folder you're running
this notebook at!

In [11]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 265
NUM_EPOCHS = 15
NUM_TRANSFORMER_BLOCKS = 15        # Number of transformer blocks.
NUM_HEADS = 8                      # Number of attention heads.
EMBEDDING_DIMS = 20                # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [2, 1]  # MLP hidden layer units, as factors related to the number of inputs.
NUM_MLP_BLOCKS = 2                 # Number of MLP blocks in the baseline model.

tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
    inputs=inputs,
    categorical_features_names=CATEGORICAL_FEATURE_NAMES,
    categorical_features_vocabulary=CATEGORICAL_FEATURES_WITH_VOCABULARY,
)

console.print(f"[bold green]Total model weights: {tabtransformer_model.count_params():,}[/bold green]")
tf.keras.utils.plot_model(
    tabtransformer_model, show_shapes=True, rankdir="LR", to_file=r'./tabtransformer.png'
)
console.print(f"[bold green]Model succeassfully generated![/bold green]")

/opt/anaconda3/envs/pyml/lib/python3.8/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


Total model weights: 293,271

Model succeassfully generated!

## Train Dataset

Generating the training dataset, to train the model.

In [12]:
train_dataset = (
    df_to_dataset(train_data, TARGET_FEATURE_NAME)
    .map(
        prepare_example,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=False,
    )
    .cache()
)

<ipython-input-8-769da26d7ea1>:345: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
/opt/anaconda3/envs/pyml/lib/python3.8/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


## Model Training

Based on the generated dataset, and the compiled model, let's then train our model.

#### <font color=red>Warning!</font>

Due to the shear size of the model, and its number o hyperparameters, please be aware that 
the generated model might take a considerable amount of time to train. This remark might be true,
regardless of the size of your training dataset.

In [61]:
history, trained_model = run_experiment(
    model=tabtransformer_model,
    dataset=train_dataset,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

Start training the model...
Epoch 1/15


/opt/anaconda3/envs/pyml/lib/python3.8/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['FLG_ACAO_EFICAZ'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


12834/12834 [==============================] - 622s 47ms/step - loss: 0.5271 - accuracy: 0.7752
Epoch 2/15
12834/12834 [==============================] - 767s 60ms/step - loss: 0.5207 - accuracy: 0.7773
Epoch 3/15
12834/12834 [==============================] - 887s 69ms/step - loss: 0.5208 - accuracy: 0.7774
Epoch 4/15
12834/12834 [==============================] - 733s 57ms/step - loss: 0.5208 - accuracy: 0.7774
Epoch 5/15
12834/12834 [==============================] - 772s 60ms/step - loss: 0.5207 - accuracy: 0.7774
Epoch 6/15
12834/12834 [==============================] - 777s 61ms/step - loss: 0.5208 - accuracy: 0.7774
Epoch 7/15
12834/12834 [==============================] - 817s 64ms/step - loss: 0.5208 - accuracy: 0.7775
Epoch 8/15
12834/12834 [==============================] - 844s 66ms/step - loss: 0.5209 - accuracy: 0.7774
Epoch 9/15
12834/12834 [==============================] - 898s 70ms/step - loss: 0.5207 - accuracy: 0.7775
Epoch 10/15
12834/12834 [=======================